In [1]:
from __future__ import division

In [2]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp
import urllib

In [3]:
Gb = 1000000000

In [4]:
#store = pd.HDFStore('access_log.h5')

In [5]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [6]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [7]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [8]:
def process_juniper_line(line):
    result = []
    line = hp.remove_duplicate_spaces(line).split(' ')
    result.append(line[0] + ' ' + line[1] + ' ' + line[2])
    result.append(line[3])
    result.append(line[4] + ' ' + line[5])
    result.append(line[6])
    result.append(line[8] + ' ' + line[9])
    for i in range(10, 13):
        result.append(line[i])
    result.append(line[14] + ' ' + line[15])    
    result.append(line[16] + ' ' + line[17])
    for i in range(18, 25):
        result.append(line[i])
    result.append(line[26] + ' ' + line[27]) 
    result.append(line[28] + ' ' + line[29]) 
    for i in range(30, len(line) - 4):
        result.append(line[i])
    result.append(line[32] + ' ' + line[33] + ' ' + line[34] + ' ' + line[35])
    return result

In [9]:
#squid = sq.read_squid_log('access.log')

In [10]:
#squid.tail()

In [11]:
#safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [12]:
#access = squid[squid['URL'].str.contains(safe_websites) == False]

In [13]:
#sum(squid.bytes)/Gb

In [72]:
def read_syslog_juniper(filename, filter=None):
    result = []
    
    #if (filename)[-3:] == '.gz':
    #    f = hp.open_gz(filename)
    #elif (filename)[-6:] == 'tar.gz':
    #    f = hp.open_tar_gz(filename)
    #else:
    #    f = hp.open_txt(filename)
    
    #f = f.split('\n')
    
    df = pd.read_table(filename, sep=' {1,}|"', engine='python', index_col=False, header=None, parse_dates=[[0, 1, 2], [9,10]])
    
    # restructure the dataframe
    i = [0]
    i.extend(range(2, 8))
    i.extend([1])
    i.extend(range(9, 25))
    df = df.iloc[:, i]
    
    #return df
    return df

In [73]:
URL = urllib.urlopen('https://doc-04-0c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k2437uo2luabbjlvpq4q5mvnshcmb5m7/1442995200000/13625217592848274312/*/0B9C3OkBK2B-JRlBkYXI4UXo5RVU?e=download')

In [74]:
jp = read_syslog_juniper(URL)

In [75]:
jp.head()

,0_1_2,3,4,5,6,7,8,9_10,12,13,...,18,19,20,21,22,23,24,25,26,27
0,2015-09-20 03:47:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,start_time=,2015-09-20 03:59:04,duration=0,policy_id=6,...,dst,zone=Untrust,action=Deny,sent=0,rcvd=28,src=172.20.78.90,dst=192.168.137.42,src_port=139,dst_port=2539,session_id=0
1,2015-09-20 03:47:05,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,start_time=,2015-09-20 03:58:05,duration=61,policy_id=397,...,dst,zone=HK-DMZ,action=Permit,sent=93,rcvd=0,src=10.20.47.254,dst=172.24.66.31,src_port=161,dst_port=37738,src-xlated
2,2015-09-20 03:47:05,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,start_time=,2015-09-20 03:58:04,duration=62,policy_id=587,...,dst,zone=HK-DMZ,action=Permit,sent=90,rcvd=0,src=172.24.66.31,dst=172.20.68.253,src_port=47176,dst_port=161,src-xlated
3,2015-09-20 03:47:05,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,start_time=,2015-09-20 03:58:05,duration=61,policy_id=505,...,dst,zone=HK-LAN,action=Permit,sent=92,rcvd=0,src=172.24.66.31,dst=172.20.78.2,src_port=42377,dst_port=161,src-xlated
4,2015-09-20 03:47:05,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,start_time=,2015-09-20 03:58:05,duration=61,policy_id=505,...,dst,zone=HK-LAN,action=Permit,sent=92,rcvd=0,src=172.24.66.31,dst=172.20.78.2,src_port=37738,dst_port=161,src-xlated
